In [53]:
import pandas as pd
import pickle
import re
from collections import defaultdict

In [131]:
census_blocks_df = pd.read_csv('../data/block_shapes/CensusBlockTIGER2010.csv')

census_blocks_df.shape



(46357, 8)

In [128]:
census_blocks_df[census_blocks.NAME10=='Block 2010'].the_geom[0]


'MULTIPOLYGON (((-87.6290679994106 41.769086000071525, -87.6290529997898 41.7688739999653, -87.62900799929758 41.76726800005111, -87.62949800027731 41.76725900029384, -87.62982000079478 41.76725400012672, -87.62985200001118 41.76839999946078, -87.62986200062038 41.7688819996305, -87.62985900013145 41.76908200019735, -87.62969199933941 41.76908500030279, -87.6290679994106 41.769086000071525)))'

In [115]:
(census_blocks_df)
# census_blocks_df.shape
census_blocks.TRACT_BLOC.count()


46357

In [18]:
block_dict = {}
for index, row in census_blocks_df.iterrows():
    block_coord = row['the_geom'].split(" ")
    lat_pat = re.compile('[^-\d][\d]+[.][\d]+')
    lats = lat_pat.findall(str(block_coord))
    long_pat = re.compile('[-][\d]+[.][\d]+')
    longs = long_pat.findall(str(block_coord))
    float_lats = [float(x[1:]) for x in lats]
    avg_lat = sum(float_lats)/len(float_lats)
    float_longs = [float(x) for x in longs]
    avg_long = sum(float_longs)/len(float_longs)

    block_dict[row['NAME10']] = [avg_lat, avg_long]

In [99]:
len(block_dict)



651

## Coffee Shops


In [111]:
import math

In [76]:
with open('../data/yelp/yelp_coffee_shop_dict.pk', 'rb') as read_file:
    coffee_shops = pickle.load(read_file)

In [77]:
block_ex = block_dict['Block 4007']
shop_ex = coffee_shops['f5p7scO1Q9UdaGbYe9bjFg']

In [78]:
block_coffee_dict = defaultdict(int)
for key_b, value_b in block_dict.items():
    for key_c, value_c in coffee_shops.items():
        lat_block = value_b[0]
        lon_block = value_b[1]
        lat_shop = value_c[2]
        lon_shop = value_c[3]
        dist = math.sqrt((lat_block -lat_shop)**2 + (lon_block - lon_shop)**2)
        if dist < .035:
            block_coffee_dict[key_b] += 1


In [98]:
#Here are the coffeeshops within a 2 mile radius of each block
len(block_coffee_dict)


514

In [51]:
dist = math.sqrt((41.963673-41.964346)**2 + (-87.786847 - -87.751445)**2)

In [52]:
dist

0.03540839636300031

lat_long distance of .035 is about 2 miles

## Businesses

In [59]:
df_businesses = pd.read_csv('../data/Business_Licenses_-_Current_Active.csv')




In [129]:
df_businesses.shape



(61799, 34)

In [69]:
business_dict = {}
for i,row in df_businesses.iterrows():
    business_dict[row['LICENSE ID']] = [row.LATITUDE, row.LONGITUDE]

In [132]:
len(business_dict)

61799

In [80]:
block_bus_dict = defaultdict(int)
for key_b, value_b in block_dict.items():
    for key_bus, value_bus in business_dict.items():
        lat_block = value_b[0]
        lon_block = value_b[1]
        lat_bus = value_bus[0]
        lon_bus = value_bus[1]
        dist = math.sqrt((lat_block -lat_bus)**2 + (lon_block - lon_bus)**2)
        if dist < .035:
            block_bus_dict[key_b] += 1

In [97]:
len(block_bus_dict)




651

## Merge coffee shop count and business count

In [83]:
block_bus_df = pd.DataFrame.from_dict(block_bus_dict, orient='index')

In [89]:
block_bus_df.rename(columns = {0:'business_count'}, inplace=True)

In [92]:
block_coffee_df = pd.DataFrame.from_dict(block_coffee_dict, orient='index')
block_coffee_df.rename(columns = {0:'coffeeshop_count'}, inplace=True)

In [94]:
df_block_bus = pd.merge(block_coffee_df, block_bus_df, left_index=True, right_index=True)



In [130]:
df_block_bus




,coffeeshop_count,business_count
Block 2010,21,2334
Block 3007,5,1679
Block 3013,113,5602
Block 4019,6,861
Block 4016,27,2671
Block 4000,14,2058
Block 2003,140,5647
Block 2002,24,2633
Block 3019,7,1611
Block 3005,91,5297
